# Exploration 14
---
## 아이유 팬이 좋아할 만한 다른 아티스트 찾기

## 1. 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남기기.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꾸기.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어오기.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
df1 = ratings.set_index("movie_id")

In [7]:
ratings2 = ratings.join(movies.set_index('movie_id'), on='movie_id')

In [8]:
using_cols = ['user_id', 'movie_id', 'counts', 'title']

In [9]:
ratings2 = ratings2[using_cols]
ratings2.head(10)

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
5,1,1197,3,"Princess Bride, The (1987)"
6,1,1287,5,Ben-Hur (1959)
7,1,2804,5,"Christmas Story, A (1983)"
8,1,594,4,Snow White and the Seven Dwarfs (1937)
9,1,919,4,"Wizard of Oz, The (1939)"


## 2. 데이터 분석

- ratings에 있는 유니크한 영화 개수
- ratings 에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [10]:
# 영화 개수
ratings2['movie_id'].nunique()

3628

In [11]:
# 유저 수
ratings2['user_id'].nunique()

6039

In [12]:
movie_count = ratings2.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [13]:
# 유저별 몇 명의 영화를 보는 지에 대한 통계
user_count = ratings2.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## 3. 선호하는 영화 5가지를 골라 ratings에 추가

In [14]:
my_favorite = ['Men in Black (1997)','Matrix, The (1999)','Star Wars: Episode VI - Return of the Jedi (1983)','E.T. the Extra-Terrestrial (1982)','Toy Story (1995)']
my_favorite_id = ['2297','2434','2716','2102','2000']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하자.
my_movielist = pd.DataFrame({'user_id': ['minkyun']*5, 'movie_id' : my_favorite_id,'counts':[5]*5, 'title': my_favorite})

if not ratings2.isin({'user_id':['minkyun']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings2 = ratings2.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가.

ratings2.tail(10)       # 잘 추가되었는지 확인.

,user_id,movie_id,counts,title
1000203,6040,1090,3,Platoon (1986)
1000205,6040,1094,5,"Crying Game, The (1992)"
1000206,6040,562,5,Welcome to the Dollhouse (1995)
1000207,6040,1096,4,Sophie's Choice (1982)
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982)
0,minkyun,2297,5,Men in Black (1997)
1,minkyun,2434,5,"Matrix, The (1999)"
2,minkyun,2716,5,Star Wars: Episode VI - Return of the Jedi (1983)
3,minkyun,2102,5,E.T. the Extra-Terrestrial (1982)
4,minkyun,2000,5,Toy Story (1995)


## 4. CSR matrix 생성

In [15]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings2['user_id'].unique()
movie_unique = ratings2['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자이다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [16]:
# 인덱싱이 잘 되었는지 확인. 
print(user_to_idx['minkyun'])
print(movie_to_idx['Men in Black (1997)'])

6039
175


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 에서 참고할 수 있다.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 보자. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거. 
temp_user_data = ratings2['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings2):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings2['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 준다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 주자. 
temp_movie_data = ratings2['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings2):
    print('title column indexing OK!!')
    ratings2['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings2

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
0,6039,2297,5,175
1,6039,2434,5,124
2,6039,2716,5,64
3,6039,2102,5,26


In [18]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings2['user_id'].nunique()
num_movie = ratings2['title'].nunique()

csr_data = csr_matrix((ratings2.counts, (ratings2.user_id, ratings2.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델 구성 및 훈련

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=25, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose 해준다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/25 [00:00<?, ?it/s]

## 6. 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈려된 모델이 예측한 나의 선호도를 파악

### 1) 선호하는 영화

In [23]:
minkyun, Matrix = user_to_idx['minkyun'], movie_to_idx['Matrix, The (1999)']
minkyun_vector, Matrix_vector = als_model.user_factors[minkyun], als_model.item_factors[Matrix]

print('슝=3')

슝=3


In [24]:
minkyun_vector

array([ 0.14158677, -0.40971172,  0.77231604, -0.00914211, -0.1592182 ,
        0.428794  , -0.1295235 ,  0.18537569, -0.10385872,  0.2727913 ,
        0.15445888,  0.59562975,  0.46898818,  0.2581612 , -0.67381155,
        0.11750712, -0.5704141 , -0.39718187, -0.18119913, -0.02279573,
        0.17541769,  0.75613   ,  0.31828326, -0.49428055, -0.29748708,
        0.3973257 ,  1.055043  ,  0.73621386,  0.24202849,  0.4699721 ,
        0.26699144, -0.6788639 ,  0.59100723, -0.87513494, -0.6779426 ,
        0.11347741,  0.25590688,  0.45322242, -0.26724494, -0.18240272,
        0.68088305,  0.34147882, -0.20391175,  0.11801398, -0.89049137,
       -0.7206135 ,  0.19894582,  0.2708665 , -0.48913914,  0.35975114,
        0.1890185 , -0.18450072, -0.07409647, -0.0568486 , -0.47448725,
       -0.22437751,  0.36088434,  0.30121788, -0.05147341,  0.03569239,
        0.36375958,  0.71417373, -0.24734598, -0.7830929 ,  0.26712608,
       -0.1890063 ,  0.09243056,  0.0964978 ,  1.3714051 , -0.38

In [25]:
Matrix_vector

array([ 1.01123191e-02, -1.72124822e-02,  1.29720969e-02, -9.55387671e-03,
       -2.92892382e-03,  1.00064687e-02,  8.83549266e-03,  8.24009441e-03,
       -1.21760629e-02,  4.42326218e-02,  2.80490480e-02,  4.03170474e-02,
       -3.52975971e-04,  2.48380266e-02, -3.26271504e-02,  1.72942411e-02,
       -2.64395177e-02, -8.45297985e-03,  1.15785357e-02, -5.99881494e-03,
        3.02858893e-02,  2.11327113e-02,  1.02789197e-02, -1.82280280e-02,
        3.24250525e-03,  2.86207367e-02,  1.17970956e-02,  3.13710049e-02,
        4.26863618e-02,  4.47370745e-02,  1.53146386e-02, -1.26405563e-02,
        9.80153400e-03,  3.10666836e-03, -2.46529337e-02,  1.24367867e-02,
        1.87180173e-02, -1.57086644e-02, -1.86242945e-02,  1.75638534e-02,
        5.72623266e-03, -4.54295333e-03,  3.37129682e-02,  1.11096678e-02,
       -4.02354868e-03,  1.24353981e-02,  2.40348466e-02,  1.15697337e-02,
        2.08638539e-03,  1.49399843e-02, -1.49465576e-02, -3.03192995e-03,
       -1.96144264e-02, -

In [26]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(minkyun_vector, Matrix_vector)

0.5837185

In [27]:
Braveheart = movie_to_idx['Braveheart (1995)']
Braveheart_vector = als_model.item_factors[Braveheart]
np.dot(minkyun_vector, Braveheart_vector)

0.27498633

In [28]:
print (movies.loc[movies['title'] == 'Braveheart (1995)'])

     movie_id              title             genre
108       110  Braveheart (1995)  Action|Drama|War


## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [29]:
favorite_movie = 'Matrix, The (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(124, 1.0),
 (92, 0.77656054),
 (62, 0.67102724),
 (200, 0.5675946),
 (141, 0.54841626),
 (145, 0.5481634),
 (107, 0.51591516),
 (375, 0.48792705),
 (175, 0.45036307),
 (317, 0.42505172),
 (75, 0.4222443),
 (44, 0.39126346),
 (117, 0.36725572),
 (193, 0.3633525),
 (479, 0.34451026)]

In [30]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Terminator, The (1984)',
 'Fugitive, The (1993)',
 'Fifth Element, The (1997)',
 'Jurassic Park (1993)',
 'Face/Off (1997)',
 'Men in Black (1997)',
 'Twelve Monkeys (1995)',
 'Hunt for Red October, The (1990)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Alien (1979)',
 'Contact (1997)']

In [31]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [32]:
get_similar_movie('Matrix, The (1999)')

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Terminator, The (1984)',
 'Fugitive, The (1993)',
 'Fifth Element, The (1997)',
 'Jurassic Park (1993)',
 'Face/Off (1997)',
 'Men in Black (1997)',
 'Twelve Monkeys (1995)']

## 8. 내가 가장 좋아할 만한 영화들을 추천

In [33]:
user = user_to_idx['minkyun']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(117, 0.611336),
 (44, 0.57017756),
 (92, 0.5324728),
 (60, 0.51625264),
 (107, 0.488379),
 (22, 0.42072985),
 (50, 0.37725553),
 (160, 0.31556457),
 (4, 0.31548274),
 (322, 0.3029769),
 (62, 0.28613216),
 (670, 0.27534655),
 (87, 0.27498633),
 (99, 0.26587522),
 (200, 0.26205027),
 (5, 0.25703767),
 (317, 0.25694093),
 (120, 0.25455558),
 (110, 0.24128345),
 (141, 0.23880862)]

In [34]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Terminator 2: Judgment Day (1991)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Jurassic Park (1993)',
 'Back to the Future (1985)',
 'Toy Story 2 (1999)',
 'Forrest Gump (1994)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Total Recall (1990)',
 'Galaxy Quest (1999)',
 'Braveheart (1995)',
 'American Beauty (1999)',
 'Terminator, The (1984)',
 'Princess Bride, The (1987)',
 'Twelve Monkeys (1995)',
 'Raiders of the Lost Ark (1981)',
 'Groundhog Day (1993)',
 'Fugitive, The (1993)']

In [35]:
# 추천에 기여한 정도
Xman = movie_to_idx['X-Men (2000)']
explain = als_model.explain(user, csr_data, itemid=Xman)

In [36]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.1123100775760715),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.06607174087700818),
 ('Men in Black (1997)', 0.04374438838721238),
 ('Toy Story (1995)', -0.005706328812545284),
 ('E.T. the Extra-Terrestrial (1982)', -0.02411263224834979)]

# 회고

## 1. 모델에 대한 평가

이번 프로젝트는 음악 추천 알고리즘을 설계한 경험을 바탕으로 영화 추천 시스템 모델을 설계하는 프로젝트였다. 기본적으로 방식이 거의 동일 했지만, 음악 추천에서는 기준을 플레이 횟수로 하였으나 영화 추천에서는 별점을 시청 횟수로 가정하여 프로젝트를 진행하였다. <br>
결과를 살펴보면 선호영화로 추가하였던 `Matrix`의 경우, 0.5837 정도로 애매하긴 하지만 0.5는 넘는 값을 보여주고 있다. `epoch`을 바꿔가며 시행하였으나 15 ~ 25 시행에서는 0.58값을 크게 벗어나지 않았다. 추천 영화의 장르는 `genre`을 제거 하여 ratings 데이타셋을 확인해보면 Sci-fi가 많았다. 이에 반해 `Braveheart`에 대한 선호도 값을 살펴보면 0.27 정도로 장르가 다소 다르기 때문에 낮은 값이 측정된 것으로 보인다.

## 2. 후기

Pandas 의 숙련도가 얼마나 중요한 지 새삼 느끼게 되었다. 기존 노드의 데이터 셋과 달리 이번에는 ratings, movies 두가지 데이터에서, `movie_id` column 을 바탕으로 합쳐야 수월하게 프로젝트가 진행이 가능했는데, 이 부분에 대한 숙련도가 미흡해서 약간 애를 먹었다. 처음 진행해보는 추천시스템은 결과가 충분히 납득되는 것 같아 프로젝트를 진행함에 있어서도 만족스러웠던 것 같다.

## 3. Reference

https://ahnty0122.tistory.com/51